# Cleaning CRM Data: Finding and Grouping Duplicate Customer Records

## The Challenge

Every CRM database accumulates duplicates over time. The same customer appears multiple times with slight variations:

| Record # | Name | Source |
|----------|------|--------|
| 1001 | John Smith | Web form |
| 1042 | Jon Smith | Sales import |
| 1089 | JOHN SMITH | Call center |
| 1156 | John A. Smith | Partner API |

These are all the same person, but your database treats them as four different customers. This causes:

- **Fragmented customer history**: Purchase records split across accounts
- **Wasted marketing spend**: Same person receives multiple emails
- **Poor customer experience**: "You're not in our system" when they clearly are
- **Inaccurate analytics**: Customer count inflated, lifetime value understated

Manual deduplication doesn't scale. A database with 100,000 customers has nearly 5 billion possible pairs to compare.

## What You'll Learn

In this tutorial, we'll build an automated deduplication pipeline:

1. **Understanding name variations**: Why "Jon" matches "John" but not "Jane"
2. **Choosing the right algorithm**: Jaro-Winkler vs. Levenshtein vs. Phonetic
3. **Using `find_duplicates()`**: FuzzyRust's built-in clustering function
4. **Handling edge cases**: Middle initials, suffixes (Jr., III), nicknames
5. **Production strategies**: Blocking, thresholds, and human review workflows

By the end, you'll have a pipeline that identifies duplicate groups automatically.

## Setup

### Prerequisites

1. Install FuzzyRust: `pip install fuzzyrust`
2. Download a name dataset:
   - **Option A (Recommended)**: NC Voter Registration data
     - Go to: https://www.ncsbe.gov/results-data/voter-registration-data
     - Download a county file (any county, ~10-50K records)
     - Extract and note the path to the `.txt` file
   - **Option B**: Use the synthetic data provided below

The NC Voter data is public record and contains realistic name variations—perfect for testing deduplication.

In [ ]:
import fuzzyrust as fr
import csv
from pathlib import Path
from collections import defaultdict

print("FuzzyRust loaded successfully")

In [ ]:
def load_nc_voter_data(filepath: str, limit: int = 1000) -> list[dict]:
    """
    Load names from NC Voter Registration data.
    
    The file is tab-separated with columns including:
    last_name, first_name, middle_name, name_suffix_lbl
    """
    records = []
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        reader = csv.DictReader(f, delimiter='\t')
        for i, row in enumerate(reader):
            if i >= limit:
                break
            # Combine name parts
            parts = [
                row.get('first_name', '').strip(),
                row.get('middle_name', '').strip(),
                row.get('last_name', '').strip(),
            ]
            suffix = row.get('name_suffix_lbl', '').strip()
            if suffix:
                parts.append(suffix)
            
            full_name = ' '.join(p for p in parts if p)
            if full_name:
                records.append({
                    'id': i,
                    'name': full_name,
                    'first': row.get('first_name', '').strip(),
                    'last': row.get('last_name', '').strip(),
                })
    return records


# Try to load NC Voter data
DATA_PATH = Path("data/ncvoter_Statewide.txt")  # Adjust path as needed

if DATA_PATH.exists():
    customers = load_nc_voter_data(str(DATA_PATH), limit=2000)
    print(f"Loaded {len(customers)} records from NC Voter data")
else:
    print(f"Dataset not found at {DATA_PATH}")
    print("Using synthetic CRM data with realistic name variations...")
    print()
    
    # Synthetic data with intentional duplicates
    customers = [
        # Group 1: John Smith variations
        {"id": 1001, "name": "John Smith", "first": "John", "last": "Smith"},
        {"id": 1042, "name": "Jon Smith", "first": "Jon", "last": "Smith"},
        {"id": 1089, "name": "JOHN SMITH", "first": "JOHN", "last": "SMITH"},
        {"id": 1156, "name": "John A. Smith", "first": "John", "last": "Smith"},
        {"id": 1201, "name": "John Smith Jr.", "first": "John", "last": "Smith"},
        {"id": 1245, "name": "Johnny Smith", "first": "Johnny", "last": "Smith"},
        
        # Group 2: Jane Doe variations
        {"id": 2001, "name": "Jane Doe", "first": "Jane", "last": "Doe"},
        {"id": 2055, "name": "Jane M. Doe", "first": "Jane", "last": "Doe"},
        {"id": 2103, "name": "Jayne Doe", "first": "Jayne", "last": "Doe"},
        {"id": 2178, "name": "Jane Doe-Smith", "first": "Jane", "last": "Doe-Smith"},
        
        # Group 3: Robert Johnson variations
        {"id": 3001, "name": "Robert Johnson", "first": "Robert", "last": "Johnson"},
        {"id": 3067, "name": "Bob Johnson", "first": "Bob", "last": "Johnson"},
        {"id": 3124, "name": "Rob Johnson", "first": "Rob", "last": "Johnson"},
        {"id": 3189, "name": "Robert B. Johnson", "first": "Robert", "last": "Johnson"},
        {"id": 3256, "name": "Robt Johnson", "first": "Robt", "last": "Johnson"},
        
        # Group 4: William Davis variations  
        {"id": 4001, "name": "William Davis", "first": "William", "last": "Davis"},
        {"id": 4078, "name": "Bill Davis", "first": "Bill", "last": "Davis"},
        {"id": 4134, "name": "Will Davis", "first": "Will", "last": "Davis"},
        {"id": 4201, "name": "Wm Davis", "first": "Wm", "last": "Davis"},
        {"id": 4267, "name": "Billy Davis", "first": "Billy", "last": "Davis"},
        
        # Group 5: Catherine Wilson variations
        {"id": 5001, "name": "Catherine Wilson", "first": "Catherine", "last": "Wilson"},
        {"id": 5089, "name": "Katherine Wilson", "first": "Katherine", "last": "Wilson"},
        {"id": 5145, "name": "Kate Wilson", "first": "Kate", "last": "Wilson"},
        {"id": 5212, "name": "Cathy Wilson", "first": "Cathy", "last": "Wilson"},
        {"id": 5278, "name": "Katie Wilson", "first": "Katie", "last": "Wilson"},
        
        # Unique records (no duplicates)
        {"id": 6001, "name": "Michael Brown", "first": "Michael", "last": "Brown"},
        {"id": 6050, "name": "Sarah Miller", "first": "Sarah", "last": "Miller"},
        {"id": 6100, "name": "David Garcia", "first": "David", "last": "Garcia"},
        {"id": 6150, "name": "Emily Martinez", "first": "Emily", "last": "Martinez"},
        {"id": 6200, "name": "James Anderson", "first": "James", "last": "Anderson"},
        {"id": 6250, "name": "Elizabeth Thomas", "first": "Elizabeth", "last": "Thomas"},
        {"id": 6300, "name": "Christopher Lee", "first": "Christopher", "last": "Lee"},
        {"id": 6350, "name": "Jennifer Taylor", "first": "Jennifer", "last": "Taylor"},
    ]
    
print(f"\nTotal records: {len(customers)}")
print("\nSample records:")
for c in customers[:8]:
    print(f"  ID {c['id']}: {c['name']}")

## Understanding Name Variations

Before diving into algorithms, let's understand the types of variations we're dealing with:

| Type | Example | Challenge |
|------|---------|----------|
| **Typos** | Jon → John | Missing character |
| **Case** | JOHN → John | Uppercase entry |
| **Middle names** | John Smith → John A. Smith | Extra information |
| **Suffixes** | John Smith → John Smith Jr. | Generational suffix |
| **Nicknames** | Robert → Bob, William → Bill | Completely different strings! |
| **Phonetic** | Catherine → Katherine | Same sound, different spelling |
| **Abbreviations** | William → Wm, Robert → Robt | Historical conventions |

The hardest cases are **nicknames** because they share no characters. "Robert" and "Bob" have zero overlap, yet they're the same person.

Let's see how different algorithms handle these:

In [ ]:
# Test different types of name variations
test_pairs = [
    ("John Smith", "Jon Smith", "Typo"),
    ("John Smith", "JOHN SMITH", "Case"),
    ("John Smith", "John A. Smith", "Middle initial"),
    ("John Smith", "John Smith Jr.", "Suffix"),
    ("Robert Johnson", "Bob Johnson", "Nickname"),
    ("Catherine Wilson", "Katherine Wilson", "Phonetic"),
    ("William Davis", "Wm Davis", "Abbreviation"),
]

print("How Different Algorithms Handle Name Variations")
print("=" * 90)
print(f"{'Name Pair':<45} {'Type':<12} {'Lev':<8} {'Jaro-W':<8} {'Soundex'}")
print("-" * 90)

for name1, name2, variation_type in test_pairs:
    # Case-insensitive comparisons
    lev = fr.levenshtein_similarity_ci(name1, name2)
    jw = fr.jaro_winkler_similarity_ci(name1, name2)
    
    # Phonetic - compare last names only (more meaningful)
    last1 = name1.split()[-1]
    last2 = name2.split()[-1]
    phonetic = "Match" if fr.soundex_match(last1, last2) else "No"
    
    pair_display = f"{name1} ↔ {name2}"
    print(f"{pair_display:<45} {variation_type:<12} {lev:<8.3f} {jw:<8.3f} {phonetic}")

### Key Insight: Jaro-Winkler for Names

Notice that **Jaro-Winkler consistently scores higher** for name variations. This is because:

1. **Prefix bonus**: Names often share the same beginning ("John" and "Jon" both start with "Jo")
2. **Transposition tolerance**: Handles character swaps well
3. **Length normalization**: Works well for varying-length names

However, Jaro-Winkler still fails for nicknames ("Robert" vs "Bob"). For those, we need:
- A nickname lookup table, OR
- Phonetic matching on surnames + fuzzy matching on first names separately

## Using `find_duplicates()` for Automatic Clustering

FuzzyRust's `find_duplicates()` function does the heavy lifting:

1. Compares all pairs of strings above a similarity threshold
2. Builds a graph where edges connect similar strings
3. Finds connected components (clusters) in the graph
4. Returns groups of duplicates

This means if A matches B, and B matches C, all three end up in the same group—even if A and C don't directly match!

In [ ]:
# Extract just the names for deduplication
names = [c["name"] for c in customers]

print(f"Finding duplicates among {len(names)} names...")
print()

# Run deduplication with Jaro-Winkler
result = fr.find_duplicates(
    names,
    algorithm="jaro_winkler",
    min_similarity=0.85,      # 85% similarity threshold
    normalize="lowercase",    # Handle case variations
    method="auto"             # Automatically choose best method
)

print(f"Results:")
print(f"  Duplicate groups found: {len(result.groups)}")
print(f"  Total duplicates: {result.total_duplicates}")
print(f"  Unique records: {len(result.unique)}")

In [ ]:
# Display the duplicate groups with original IDs
name_to_id = {c["name"]: c["id"] for c in customers}

print("\nDuplicate Groups:")
print("=" * 60)

for i, group in enumerate(result.groups, 1):
    print(f"\n📁 Group {i} ({len(group)} records):")
    for name in group:
        record_id = name_to_id.get(name, "?")
        print(f"   ID {record_id}: {name}")

### Adjusting the Threshold

The `min_similarity` threshold is critical:

- **Too high (0.95+)**: Misses legitimate duplicates ("Jon" won't match "John")
- **Too low (0.70-)**: Creates false positives ("John Smith" might match "Jane Smith")

Let's see how different thresholds affect our results:

In [ ]:
print("Effect of Threshold on Deduplication")
print("=" * 60)
print(f"{'Threshold':<12} {'Groups':<10} {'Duplicates':<12} {'Unique'}")
print("-" * 60)

for threshold in [0.95, 0.90, 0.85, 0.80, 0.75, 0.70]:
    result = fr.find_duplicates(
        names,
        algorithm="jaro_winkler",
        min_similarity=threshold,
        normalize="lowercase"
    )
    print(f"{threshold:<12.2f} {len(result.groups):<10} {result.total_duplicates:<12} {len(result.unique)}")

## Handling Nicknames: A Two-Stage Approach

Nicknames are the hardest problem. "Robert" and "Bob" have essentially zero character overlap. 

There are two approaches:

### Approach 1: Nickname Lookup Table
Maintain a dictionary mapping nicknames to canonical names:
```python
nicknames = {
    "bob": "robert", "rob": "robert", "robbie": "robert",
    "bill": "william", "will": "william", "wm": "william",
    "kate": "catherine", "cathy": "catherine", "katie": "catherine",
}
```

### Approach 2: Two-Stage Matching
1. **Stage 1**: Match on last name (must be similar)
2. **Stage 2**: For matches, check if first names are similar OR known nickname pairs

In [ ]:
# Common nickname mappings
NICKNAMES = {
    # Robert variants
    "bob": "robert", "rob": "robert", "robbie": "robert", "bobby": "robert", "robt": "robert",
    # William variants
    "bill": "william", "will": "william", "wm": "william", "billy": "william", "willie": "william",
    # Catherine/Katherine variants
    "kate": "catherine", "cathy": "catherine", "katie": "catherine", 
    "kathy": "katherine", "kat": "catherine",
    # John variants
    "jon": "john", "johnny": "john", "jack": "john",
    # James variants
    "jim": "james", "jimmy": "james", "jamie": "james",
    # Elizabeth variants
    "liz": "elizabeth", "beth": "elizabeth", "betty": "elizabeth", "lizzy": "elizabeth",
    # Michael variants
    "mike": "michael", "mikey": "michael", "mick": "michael",
    # Richard variants
    "rick": "richard", "dick": "richard", "rich": "richard", "ricky": "richard",
}

def normalize_first_name(name: str) -> str:
    """Normalize first name using nickname table."""
    name_lower = name.lower().strip()
    return NICKNAMES.get(name_lower, name_lower)

def enhanced_name_match(name1: str, name2: str, threshold: float = 0.85) -> tuple[bool, float]:
    """
    Check if two names match, considering nicknames.
    
    Returns: (is_match, confidence_score)
    """
    # Split into first and last names
    parts1 = name1.split()
    parts2 = name2.split()
    
    if len(parts1) < 2 or len(parts2) < 2:
        # Fall back to full name comparison
        score = fr.jaro_winkler_similarity_ci(name1, name2)
        return score >= threshold, score
    
    first1, last1 = parts1[0], parts1[-1]
    first2, last2 = parts2[0], parts2[-1]
    
    # Last names must match well
    last_score = fr.jaro_winkler_similarity_ci(last1, last2)
    if last_score < 0.85:
        return False, last_score
    
    # Check first names
    first_score = fr.jaro_winkler_similarity_ci(first1, first2)
    
    # Check nickname equivalence
    norm1 = normalize_first_name(first1)
    norm2 = normalize_first_name(first2)
    nickname_match = (norm1 == norm2)
    
    # Calculate combined score
    if nickname_match:
        combined_score = (0.5 + last_score) / 1.5  # Boost for nickname match
        return True, combined_score
    elif first_score >= threshold:
        combined_score = (first_score + last_score) / 2
        return combined_score >= threshold, combined_score
    
    return False, (first_score + last_score) / 2

# Test the enhanced matching
print("Enhanced Name Matching with Nickname Support")
print("=" * 70)

test_pairs = [
    ("Robert Johnson", "Bob Johnson"),
    ("William Davis", "Bill Davis"),
    ("Catherine Wilson", "Kate Wilson"),
    ("John Smith", "Jon Smith"),
    ("John Smith", "Jane Smith"),  # Different person
]

for name1, name2 in test_pairs:
    match, score = enhanced_name_match(name1, name2)
    status = "✓ Match" if match else "✗ No match"
    print(f"{name1:<20} ↔ {name2:<20} → {status} ({score:.3f})")

## Phonetic Matching for Similar-Sounding Names

Some names sound the same but are spelled differently:
- Catherine / Katherine
- Steven / Stephen  
- Smith / Smyth

Phonetic algorithms encode names by how they sound. FuzzyRust provides:
- **Soundex**: Classic algorithm, 4-character codes
- **Metaphone**: More accurate, variable-length codes

In [ ]:
# Phonetic matching examples
phonetic_pairs = [
    ("Catherine", "Katherine"),
    ("Steven", "Stephen"),
    ("Smith", "Smyth"),
    ("Meyer", "Mayer"),
    ("Thompson", "Thomson"),
    ("Schneider", "Snyder"),
    ("John", "Jane"),  # Different - should NOT match
]

print("Phonetic Matching: Soundex vs Metaphone")
print("=" * 80)
print(f"{'Name 1':<15} {'Name 2':<15} {'Soundex 1':<10} {'Soundex 2':<10} {'Match?'}")
print("-" * 80)

for n1, n2 in phonetic_pairs:
    s1 = fr.soundex(n1)
    s2 = fr.soundex(n2)
    match = "✓" if s1 == s2 else "✗"
    print(f"{n1:<15} {n2:<15} {s1:<10} {s2:<10} {match}")

## Production Pipeline: Blocking + Detailed Comparison

For large datasets (100K+ records), comparing every pair is too slow. 

**Blocking** reduces the search space by only comparing records that share a common attribute:

1. **First letter of last name**: "Smith" only compared to other "S" names
2. **Soundex code**: Group phonetically similar names
3. **First 3 characters**: Quick pre-filter

This reduces O(n²) comparisons to O(n × block_size).

In [ ]:
def create_blocks(records: list[dict], method: str = "soundex_last") -> dict:
    """
    Group records into blocks for efficient comparison.
    
    Args:
        records: List of customer records with 'name' and 'last' fields
        method: Blocking method - 'soundex_last', 'first_letter', or 'first_3'
    
    Returns:
        Dict mapping block keys to lists of records
    """
    blocks = defaultdict(list)
    
    for record in records:
        last_name = record.get('last', record['name'].split()[-1])
        
        if method == "soundex_last":
            key = fr.soundex(last_name)
        elif method == "first_letter":
            key = last_name[0].upper() if last_name else "?"
        elif method == "first_3":
            key = last_name[:3].lower() if len(last_name) >= 3 else last_name.lower()
        else:
            key = "all"  # No blocking
        
        blocks[key].append(record)
    
    return dict(blocks)

# Create blocks
blocks = create_blocks(customers, method="soundex_last")

print(f"Blocking Statistics (Soundex on Last Name)")
print(f"=" * 50)
print(f"Total records: {len(customers)}")
print(f"Number of blocks: {len(blocks)}")
print(f"")

# Show block sizes
print("Largest blocks:")
sorted_blocks = sorted(blocks.items(), key=lambda x: len(x[1]), reverse=True)
for key, records in sorted_blocks[:5]:
    sample_names = [r['last'] for r in records[:3]]
    print(f"  Block '{key}': {len(records)} records (e.g., {', '.join(sample_names)})")

In [ ]:
def find_duplicates_with_blocking(
    records: list[dict],
    blocking_method: str = "soundex_last",
    threshold: float = 0.85
) -> list[list[dict]]:
    """
    Find duplicates using blocking + detailed comparison.
    
    Returns list of duplicate groups (each group is a list of records).
    """
    blocks = create_blocks(records, blocking_method)
    all_groups = []
    
    for block_key, block_records in blocks.items():
        if len(block_records) < 2:
            continue
        
        # Run deduplication within this block
        names = [r["name"] for r in block_records]
        result = fr.find_duplicates(
            names,
            algorithm="jaro_winkler",
            min_similarity=threshold,
            normalize="lowercase"
        )
        
        # Map names back to full records
        name_to_record = {r["name"]: r for r in block_records}
        for group in result.groups:
            record_group = [name_to_record[name] for name in group]
            all_groups.append(record_group)
    
    return all_groups

# Run blocked deduplication
duplicate_groups = find_duplicates_with_blocking(
    customers,
    blocking_method="soundex_last",
    threshold=0.85
)

print(f"\nBlocked Deduplication Results")
print(f"=" * 50)
print(f"Duplicate groups found: {len(duplicate_groups)}")

for i, group in enumerate(duplicate_groups, 1):
    print(f"\nGroup {i}:")
    for record in group:
        print(f"  ID {record['id']}: {record['name']}")

## Human Review Workflow

In production, automated deduplication should flag candidates for human review rather than auto-merging. Here's a simple workflow:

In [ ]:
def generate_review_report(duplicate_groups: list, min_confidence: float = 0.9) -> dict:
    """
    Generate a human review report for duplicate candidates.
    
    Returns:
        Dict with 'auto_merge' (high confidence) and 'review' (needs human eyes)
    """
    auto_merge = []
    needs_review = []
    
    for group in duplicate_groups:
        if len(group) < 2:
            continue
        
        # Calculate pairwise similarity for the group
        names = [r["name"] for r in group]
        similarities = []
        for i, n1 in enumerate(names):
            for n2 in names[i+1:]:
                sim = fr.jaro_winkler_similarity_ci(n1, n2)
                similarities.append(sim)
        
        avg_sim = sum(similarities) / len(similarities) if similarities else 0
        min_sim = min(similarities) if similarities else 0
        
        group_info = {
            "records": group,
            "avg_similarity": avg_sim,
            "min_similarity": min_sim,
        }
        
        # High confidence if ALL pairs are above threshold
        if min_sim >= min_confidence:
            auto_merge.append(group_info)
        else:
            needs_review.append(group_info)
    
    return {"auto_merge": auto_merge, "review": needs_review}

# Generate report
report = generate_review_report(duplicate_groups, min_confidence=0.92)

print("Deduplication Review Report")
print("=" * 60)
print(f"\n✅ AUTO-MERGE ({len(report['auto_merge'])} groups):")
print("   High confidence - can be merged automatically")

for group_info in report['auto_merge']:
    print(f"\n   Group (avg: {group_info['avg_similarity']:.1%}):")
    for r in group_info['records']:
        print(f"     - {r['name']} (ID: {r['id']})")

print(f"\n⚠️  NEEDS REVIEW ({len(report['review'])} groups):")
print("   Lower confidence - human verification recommended")

for group_info in report['review']:
    print(f"\n   Group (avg: {group_info['avg_similarity']:.1%}, min: {group_info['min_similarity']:.1%}):")
    for r in group_info['records']:
        print(f"     - {r['name']} (ID: {r['id']})")

## Summary

### Key Takeaways

1. **Use Jaro-Winkler for names**: Its prefix bonus works well with first/last name patterns.

2. **Normalize aggressively**: Always use case-insensitive matching; consider removing punctuation.

3. **Handle nicknames explicitly**: No algorithm will match "Robert" to "Bob"—use a lookup table.

4. **Use blocking for scale**: Don't compare every pair; group by last name soundex or similar.

5. **Implement human review**: Auto-merge high-confidence matches; flag others for review.

6. **Tune thresholds carefully**:
   - 0.90+: Conservative, few false positives
   - 0.85: Good balance
   - 0.80-: Aggressive, may need more review

### When to Use This Approach

✅ **Good for:**
- CRM deduplication
- Patient record matching
- Customer data cleaning
- Voter roll maintenance

❌ **Consider alternatives for:**
- Multi-attribute matching (add address, DOB) → use `SchemaIndex`
- Very large datasets (10M+) → consider probabilistic matching
- Real-time matching → pre-compute blocks, use indices

In [ ]:
# Quick reference: Complete deduplication pipeline
print("Quick Reference: Complete Deduplication Pipeline")
print("=" * 50)
print("""
1. LOAD DATA
   records = load_customer_data("customers.csv")

2. EXTRACT NAMES
   names = [r["name"] for r in records]

3. FIND DUPLICATES
   result = fr.find_duplicates(
       names,
       algorithm="jaro_winkler",
       min_similarity=0.85,
       normalize="lowercase"
   )

4. REVIEW RESULTS
   for group in result.groups:
       print(f"Potential duplicates: {group}")

5. (OPTIONAL) BLOCKING FOR SCALE
   blocks = create_blocks(records, "soundex_last")
   for block in blocks.values():
       # Deduplicate within each block
""")